# Road Following by Regression - Live Demo TensorRT

In this notebook, we drive the JetBot with the ResNet18 TensorRT model.

In [5]:
import torch
device = torch.device('cuda')

## Load ResNet18 TensorRT model
First we load the ResNet18 TensorRT model.

In [2]:
import torch
import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_reg_resnet18_trt.pth'))

<All keys matched successfully>

## Preprocessing Function

Now we create a function for preprocessing image data taken by the camera.

In [3]:
import cv2
import numpy as np
import torchvision
import torchvision.transforms as transforms

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.cuda().half()
    x = x[None, ...]
    return x

## Camera Instance
Now we create a camera instance.

In [10]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

size = 224
image = widgets.Image(format='jpeg', width=size, height=size)
camera = Camera.instance(width=size, height=size)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Robot Instance
Create the robot instance to drive the motors.

In [5]:
from jetbot import Robot
import time

robot = Robot()
robot.set_motors(-0.1, 0.1)
time.sleep(0.1)
robot.stop()

## Define move_robot function
Define a function to control the motors depending on the x and y coordinates of the target point.

In [6]:
def move_robot(x, y):
    # move robot
    speed_gain = 0.3
    b = 0.3               # kind of steering gain
    x = x/size-0.5
    y = 1-y/size
    a = np.sqrt(x**2 + y**2)
    angle = np.pi/2-np.arctan2(y, x)+1E-6
    c = 0.5*a/np.sin(np.abs(angle))

    if angle >= 0:
        left_motor = speed_gain*(c+b)*np.abs(2*angle)
        right_motor = speed_gain*(c-b)*np.abs(2*angle)  
    else:
        left_motor = speed_gain*(c-b)*np.abs(np.abs(2*angle))
        right_motor = speed_gain*(c+b)*np.abs(np.abs(2*angle))
    
    left_motor = max(0, left_motor)
    right_motor = max(0, right_motor)

    robot.set_motors(left_motor, right_motor)
    
    return left_motor, right_motor

## Inference
Let's try to make an inference. This process takes for a while for the first time because it needs to load a model.

In [7]:
with torch.no_grad():
    img = camera.value
    img = preprocess(img)
    print(img.shape)
    xy = size*model_trt(img)[0].to("cpu").numpy()
    print(xy)
    x = int(xy[0])
    y = int(xy[1])
    left_motor, right_motor = move_robot(x, y)
    time.sleep(0.1)
    robot.stop()
    print(f"left_motor={left_motor}, right_motor={right_motor}")

torch.Size([1, 3, 224, 224])
[128.2 139.9]
left_motor=0.15000024812928872, right_motor=0.08301894435564738


## Run JetBot
Run JetBot with a loop.

In [11]:
t0 = time.time()

display(image)
steps = 500 # if you want to run JetBot longer, increase this value

with torch.no_grad():

    for i in range(steps):
        try:
            img = camera.value
            img = preprocess(img)

            xy = size*model_trt(img)[0].to("cpu").numpy()
            x = int(xy[0])
            y = int(xy[1])

            left_motor, right_motor = move_robot(x, y)
            
        except:
            pass

        now = time.time()
        dt = now-t0
        t0 = now
        FPS = 1/dt

        print(f"\rStep:{i+1}/{steps}, left_motor={left_motor:.3f}, right_motor={right_motor:.3f}, FPS:{FPS:.1f}", end="")

robot.stop()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Step:500/500, left_motor=0.239, right_motor=0.083, FPS:17.3

## Stop Robot & Camera

If you are done, stop the robot and the camera.

In [12]:
robot.stop()
camera.stop()